In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D

In [4]:
from tensorflow.python.client import device_lib

print(tf.config.list_physical_devices('GPU'))
print("GPU is", "available" if tf.test.is_gpu_available() else "not available")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU is available


In [3]:
# Fashion MNIST:
# https://github.com/zalandoresearch/fashion-mnist

fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images.reshape(60000, 28, 28, 1)
train_images = train_images / 255.0

test_images = test_images.reshape(10000, 28, 28, 1)
test_images = test_images / 255.0

In [4]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc') is not None and logs.get('acc') >= 0.998) :
          print("\nReached 99.8% accuracy so cancelling training!")
          self.model.stop_training = True

In [5]:
def create_model():
  # Sequential defines a sequence of layers in the nn
  model = keras.Sequential([
      # Shape of images after Conv2D layer: (26, 26)
      Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
      
      # Shape of images after MaxPooling2D: (13, 13)
      MaxPooling2D(2, 2),
      
      # The flatten layer takes the 28 by 28 matrix
      # and turn it into a single array
      Flatten(),
      Dense(128, activation='relu'),

      # The following layer has 10 units because there are
      # 10 classes. To classify multiple objects
      # we use the activation function softmax
      Dense(10, activation='softmax')
  ])
  
  model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy']
  )

  return model

In [6]:
# Create a basic model instance
model = create_model()

callbacks = myCallback()

with tf.device('/gpu:0'):
    model.fit(train_images, train_labels, epochs=15, callbacks=[callbacks])

Train on 60000 samples
Epoch 1/15
60000/60000 [==============================] - 9s 152us/sample - loss: 0.3790 - accuracy: 0.8652
Epoch 2/15
60000/60000 [==============================] - 7s 121us/sample - loss: 0.2555 - accuracy: 0.9065
Epoch 3/15
60000/60000 [==============================] - 7s 121us/sample - loss: 0.2102 - accuracy: 0.9224
Epoch 4/15
60000/60000 [==============================] - 7s 121us/sample - loss: 0.1763 - accuracy: 0.9343
Epoch 5/15
60000/60000 [==============================] - 7s 120us/sample - loss: 0.1480 - accuracy: 0.9452
Epoch 6/15
60000/60000 [==============================] - 7s 123us/sample - loss: 0.1222 - accuracy: 0.9549
Epoch 7/15
60000/60000 [==============================] - 7s 124us/sample - loss: 0.1029 - accuracy: 0.9614
Epoch 8/15
60000/60000 [==============================] - 8s 128us/sample - loss: 0.0843 - accuracy: 0.9686
Epoch 9/15
60000/60000 [==============================] - 8s 130us/sample - loss: 0.0697 - accuracy: 0.9744
Epoch

In [7]:
# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 10816)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               1384576   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 1,386,506
Trainable params: 1,386,506
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 1s 115us/sample - loss: 0.4721 - accuracy: 0.9143


[0.47211130201406776, 0.9143]